In [ ]:
# Warning control
import warnings
# Ignora avisos
warnings.filterwarnings('ignore')

# Importa as classes Agent, Task e Crew da biblioteca crewai
from crewai import Agent, Task, Crew

import os
from utils import get_openai_api_key

# Obtém a chave da API do OpenAI usando uma função utilitária
openai_api_key = get_openai_api_key()
# Define o nome do modelo OpenAI a ser usado
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Criando o agente 'planner' que planeja o conteúdo
planner = Agent(
    role="Content Planner",  # Define o papel do agente
    goal="Plan engaging and factually accurate content on {topic}",  # Define o objetivo do agente
    backstory="You're working on planning a blog article "
              "about the topic: {topic}. "
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",  # História de fundo do agente
    allow_delegation=False,  # Não permite delegação de tarefas
    verbose=True  # Ativa a saída detalhada
)

# Criando o agente 'writer' que escreve o conteúdo
writer = Agent(
    role="Content Writer",  # Define o papel do agente
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",  # Define o objetivo do agente
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",  # História de fundo do agente
    allow_delegation=False,  # Não permite delegação de tarefas
    verbose=True  # Ativa a saída detalhada
)

# Criando o agente 'editor' que edita o conteúdo
editor = Agent(
    role="Editor",  # Define o papel do agente
    goal="Edit a given blog post to align with "
         "the writing style of the organization.",  # Define o objetivo do agente
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices, "
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",  # História de fundo do agente
    allow_delegation=False,  # Não permite delegação de tarefas
    verbose=True  # Ativa a saída detalhada
)

# Criando a tarefa 'plan' para o agente 'planner'
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
        "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
        "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
        "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
                    "with an outline, audience analysis, "
                    "SEO keywords, and resources.",  # Saída esperada da tarefa
    agent=planner,  # A tarefa é atribuída ao agente 'planner'
)

# Criando a tarefa 'write' para o agente 'writer'
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
        "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
        "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
        "engaging introduction, insightful body, "
        "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
        "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
                    "in markdown format, ready for publication, "
                    "each section should have 2 or 3 paragraphs.",  # Saída esperada da tarefa
    agent=writer,  # A tarefa é atribuída ao agente 'writer'
)

# Criando a tarefa 'edit' para o agente 'editor'
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),  # Descrição da tarefa
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",  # Saída esperada da tarefa
    agent=editor  # A tarefa é atribuída ao agente 'editor'
)

# Criando a equipe com os agentes e as tarefas definidas
crew = Crew(
    agents=[planner, writer, editor],  # Lista de agentes
    tasks=[plan, write, edit],  # Lista de tarefas
    verbose=2  # Nível de detalhamento da saída
)

# Executando a equipe com o tópico "Artificial Intelligence"
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})